## Text Generation with LSTM using Lord of the Rings: The Two Towers corpus 

https://medium.com/@matthewbejtlich/text-generation-with-lstm-lord-of-the-rings-the-two-towers-99e977b8a5e8

This is the second model for my text generation with LSTM. Please see Text_Generation_Model_1.ipynb for Model 1.

## Model 2

In [1]:
import keras
keras.__version__

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


'2.1.5'

In [2]:
import numpy as np

text = open('data/Lordoftherings2.txt').read().lower()
print('Corpus length:', len(text))

Corpus length: 834860


In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 278267
Unique characters: 58
Vectorization...


In [4]:
from keras import layers
from keras.layers import Dropout

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(Dropout(0.2))
model.add(layers.LSTM(128))
model.add(Dropout(0.2))
model.add(layers.Dense(len(chars), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60, 128)           95744     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 58)                7482      
Total params: 234,810
Trainable params: 234,810
Non-trainable params: 0
_________________________________________________________________


In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

for epoch in range(1, 10):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
278267/278267 [==============================] - 341s 1ms/step - loss: 2.0497
--- Generating with seed: "m the tower at 
least, and a lot more than that from down be"
------ temperature: 0.2
m the tower at 
least, and a lot more than that from down before the mars was the still and the mars was a light the store and 































































the stall and the was the come the mars the stars at the strang the stround the mind the stall the store the 
was a 

the 
the star the stround bround the strong 
the still the more the rind the stare the mare the stround 
the stround the come the was the stround the s
------ temperature: 0.5
 the stround 
the stround the come the was the stround the stood the now and the might in the rearly. 

the hort in the stor and the come the strang of the to for out and before and were many the sile in the mround the mmars be and him of the mir his the 
is the marse, and the get the room the more 

the 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


 
bard. he was and is he was for the 
the bettind him of the roads the right to the 
more to shadow to the come was the should no before they
------ temperature: 1.0
he 
more to shadow to the come was the should no before they down and be hand ent; he he amendnils from the bilen spunclessaar. in the endwhally. i sinling and and udness. we safm. thing backndesk, more what 
to the rock, 
car mive crose on stoping from it? s, and the newn, of you will rest. thinl of bats broud me dry ashedter not guasedom of caught with the nores fores the had gefan don't that the rided of oas away that to the said. 'for them to they woul
------ temperature: 1.2
e rided of oas away that to the said. 'for them to they would to turneds to his weath ent ! they h\vut to 
sey seaked and and gon of mordor and tree only picks. in there fall. the coless, grifd outs of dorses the seaviet, and bulder to 
that from his geamet to faind my. not out of inleft from the strong.' 

'wattino stoped. 'find and at flowed us 